In [1]:
import mrcfile
import os
import numpy as np

In [2]:
files = os.listdir('test_maps/')
print(files)

for file in files:

    with mrcfile.open(f'test_maps/{file}') as mrc:
    #print(mrc.data[10,10])
        print(mrc.voxel_size)

['D3_FMN_bad.mrc', '2yif.mrc', 'D4_FMN_good.mrc', 'D2_SAMIV_good.mrc', 'tmp.mrc', '6ues.mrc', 'D10_SAMIV_bad.mrc']
(1.34375, 1.34375, 1.34375)
(0.86, 0.86, 0.86)
(1.34375, 1.34375, 1.34375)
(1.34375, 1.34375, 1.34375)
(0., 0., 0.)
(0.86, 0.86, 0.86)
(1.34375, 1.34375, 1.34375)


In [3]:
mrc = mrcfile.open('test_maps/6ues.mrc', mode='r+')
mrc.header

rec.array((154, 154, 154, 2, -77, -77, -77, 154, 154, 154, (132.44, 132.44, 132.44), (90., 90., 90.), 1, 2, 3, 0., 15.619418, 0.02387825, 1, 0, b'\x00\x00\x00\x00\x00\x00\x00\x00', b'', 0, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', (-66.22, -66.22, -66.22), b'MAP ', [68, 65,  0,  0], 0.30581287, 1, [b'EMAN 8/15/2023 10:51', b'', b'', b'', b'', b'', b'', b'', b'', b'']),
          dtype=[('nx', '<i4'), ('ny', '<i4'), ('nz', '<i4'), ('mode', '<i4'), ('nxstart', '<i4'), ('nystart', '<i4'), ('nzstart', '<i4'), ('mx', '<i4'), ('my', '<i4'), ('mz', '<i4'), ('cella', [('x', '<f4'), ('y', '<f4'), ('z', '<f4')]), ('cellb', [('alpha', '<f4'), ('beta', '<f4'), ('gamma', '<f4')]), ('mapc', '<i4'), ('mapr'

In [4]:
#for modifying the contour level using mrcfile --
#in the header, dmin, dmax, and dmean correspond to the following: 
    #dmin -- minimum pixel value 
    #dmax -- maximum pixel value 
    #dmean -- mean pixel value 
    
#these need to be set for proper scaling of data 
#we have the mean for the pixel intensity, need to calculate standard deviation 
#to set contour size, just filter for values that are outside of x number of standard deviations 
    #from the mean (0.02387825)
    
#then just check the number of voxels 

In [5]:
## pseudocode layout for this script 

## first, load two .mrc files (one is the original map, the other is an initial volume)
## second, check voxel size, cell size, and grid size and calculate the dimensions of each voxel in grid points
## check how many voxels are in the original map 
## check how many voxels in the initial volume 
## iterate through a range of contour levels for the initial volume until voxel count matches the original map
## save that contour level 

In [6]:
def get_mrc_data_headers_and_vs(mrc_filename):
    with mrcfile.open(mrc_filename) as mrc:
        return mrc.data, mrc.header, mrc.voxel_size

def calculate_voxels(data, contour_level):
    threshold = np.mean(data) + (np.std(data)*contour_level)
    contoured_data = np.where(data >= threshold, data, 0)
    return np.count_nonzero(contoured_data)
    
original_map = 'test_maps/6ues.mrc'
initial_volume = 'test_maps/D10_SAMIV_bad.mrc'
    
og_data, og_header, og_voxel_size = get_mrc_data_headers_and_vs(original_map)
iv_data, iv_header, iv_voxel_size = get_mrc_data_headers_and_vs(initial_volume)

## now let's check to make sure that grid size, voxel size, and cell size are all perfectly symmetrical 

if (og_voxel_size.x != og_voxel_size.y) or (og_voxel_size.x != og_voxel_size.z) or (og_voxel_size.y 
                                                                                    != og_voxel_size.z):
    print('original map voxel size is not symmetrical')

if (iv_voxel_size.x != iv_voxel_size.y) or (iv_voxel_size.x != iv_voxel_size.z) or (iv_voxel_size.y 
                                                                                    != iv_voxel_size.z):
    print('initial volume voxel size is not symmetrical')

if (og_header.nx != og_header.ny) or (og_header.ny != og_header.nz) or (og_header.nx != og_header.nz):
    print('original map grid size is not symmetrical')

if (iv_header.nx != iv_header.ny) or (iv_header.ny != iv_header.nz) or (iv_header.nx != iv_header.nz):
    print('initial volume grid size is not symmetrical')

if (og_header.cella.x != og_header.cella.y) or (og_header.cella.y 
                                                != og_header.cella.z) or (og_header.cella.x 
                                                                          != og_header.cella.z):
    print('original map cell size is not symmetrical')

if (iv_header.cella.x != iv_header.cella.y) or (iv_header.cella.y 
                                                != iv_header.cella.z) or (iv_header.cella.x 
                                                                          != iv_header.cella.z):
    print('initial volume cell size is not symmetrical')
    
# now let's check that length of grid points per voxel is equal to 1

og_grid_pts_pvoxel = (og_header.nx/og_header.cella.x)*og_voxel_size.x
iv_grid_pts_pvoxel = (iv_header.nx/iv_header.cella.x)*iv_voxel_size.x

if round(iv_grid_pts_pvoxel, 3) != 1.0 or round(og_grid_pts_pvoxel, 3) != 1.0:
    print('grid points per voxel is off')

# now let's calculate the number of voxels in the original map 
og_voxels = calculate_voxels(og_data, contour_level=0)
print(og_voxels)

#now for our first while loop, which will have a coarse search of contour levels 
iv_contour_level = 0
iv_voxels = 0
while iv_voxels != og_voxels:
    iv_voxels = calculate_voxels(iv_data, iv_contour_level)
    if iv_voxels < og_voxels:
        break
    iv_contour_level = iv_contour_level + 0.001

#now for the second while loop, which will have a finer search of contour levels 
iv_contour_level = iv_contour_level - 0.001
while iv_voxels != og_voxels:
    iv_voxels = calculate_voxels(iv_data, iv_contour_level)
    if iv_voxels < og_voxels:
        break
    iv_contour_level = iv_contour_level + 0.000001

47312


In [7]:
#now we will print the final contour level which we have determined to give us the same number of voxels
#as the original map
print(iv_contour_level)

0.4841589999999961


In [8]:
#however, there is a problem with this method 

#since the cell size is different, the voxel size is also different, which means that having the same number
#of voxels, still means that we have a different volume represented per voxel, angstroms cubed

#so let's try a variation where instead of getting matching numbers of voxels, we choose the contour level
#that gives the same volume, angstroms^3 

In [9]:
print(og_voxels)
print(og_voxel_size)

og_volume = og_voxels * og_voxel_size.x * og_voxel_size.y * og_voxel_size.z
iv_voxels_wanted = og_volume / (iv_voxel_size.x*iv_voxel_size.y*iv_voxel_size.z)
print(og_volume)
print(iv_voxels_wanted)
print(round(iv_voxels_wanted, 0))

iv_contour_level_volume = 0
while iv_voxels != int(round(iv_voxels_wanted, 0)):
    #print(iv_contour_level_volume)
    iv_voxels = calculate_voxels(iv_data, iv_contour_level_volume)
    #print(iv_voxels)
    if iv_voxels < int(round(iv_voxels_wanted, 0)):
        break
    iv_contour_level_volume += 0.01

iv_contour_level_volume -= 0.001
while iv_voxels != int(round(iv_voxels_wanted, 0)):
    iv_voxels = calculate_voxels(iv_data, iv_contour_level_volume)
    if iv_voxels < int(round(iv_voxels_wanted, 0)):
        break
    iv_contour_level_volume += 0.000001

47312
(0.86, 0.86, 0.86)
30093.082973691828
12402.557546906986
12403.0


In [10]:
print(iv_contour_level_volume)

4.258999999999953


In [11]:
## after a visual inspection of the corresponding volumes, the contour level based off volume
## is a much more reasonable metric 